#Week 10 Keras RNN Implementations Lab and Assignment


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Before Lab and Assignment, make sure you've put today's data to `/content/gdrive/My Drive/music_data_mining/log_mels_4classes.npz`. If not, follow the procedure:

- open the url (https://drive.google.com/file/d/19Z4dcFrCyJ51l1YpvVYh3nIrDBQNebIC/view)
- click the triangle with a plus ("Add shorcut to Drive") on the upper right corner
- navigate to the right directory `/content/gdrive/My Drive/music_data_mining/` and confirm by clicking "Add shortcut here".

For today's Lab and Assignment, you have four different types of RNN-based models: Deep RNNs, GRU, LSTM, and Bidirectional GRU. Their performances are not that high because they are pretty simple. During the lab session, change some settings below to improve their performances.
Possible parameters that you can change:
- number of units
- number of layers
- dropout rate
- lookback
- learning rate
- batch size

Let me know your best network settings!

In [4]:
import gc
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Loading Data

In [3]:
log_mels= np.load('/content/gdrive/My Drive/music_data_mining/log_mels_4classes.npz') 
X_train = log_mels['X_train']
X_test = log_mels['X_test']
X_valid = log_mels['X_valid']
y_train = log_mels['y_train']
y_test = log_mels['y_test']
y_valid = log_mels['y_valid']

In [3]:
X_train.shape

(3200, 1290, 128)

In [4]:
y_train.shape

(3200,)

##RNN Data Reshaping

In [5]:
lookback=43*2
factor=int(1290/lookback)

X_train=X_train.reshape((3200*factor, lookback, 128))

y_train=np.kron(y_train, np.ones(factor))
y_train.astype(np.int32)

array([3, 3, 3, ..., 3, 3, 3], dtype=int32)

## 1. Deep RNNs

In [ ]:
if 'model' in locals():
  del model
  gc.collect()
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, dropout=0.5),
    keras.layers.SimpleRNN(100, dropout=0.1),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=4, activation='softmax')
])

In [5]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=200, epochs=200,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
       

Epoch 1/200
120/120 [==============================] - 22s 148ms/step - loss: 1.2913 - accuracy: 0.4086 - val_loss: 1.2468 - val_accuracy: 0.4175
Epoch 2/200
120/120 [==============================] - 16s 134ms/step - loss: 1.2034 - accuracy: 0.4496 - val_loss: 1.2005 - val_accuracy: 0.4250
Epoch 3/200
120/120 [==============================] - 17s 144ms/step - loss: 1.1657 - accuracy: 0.4701 - val_loss: 1.1966 - val_accuracy: 0.4300
Epoch 4/200
120/120 [==============================] - 16s 132ms/step - loss: 1.2441 - accuracy: 0.4115 - val_loss: 1.1776 - val_accuracy: 0.4675
Epoch 5/200
120/120 [==============================] - 16s 131ms/step - loss: 1.1850 - accuracy: 0.4561 - val_loss: 1.1436 - val_accuracy: 0.4625
Epoch 6/200
120/120 [==============================] - 16s 133ms/step - loss: 1.1390 - accuracy: 0.4909 - val_loss: 1.1427 - val_accuracy: 0.4800
Epoch 7/200
120/120 [==============================] - 18s 152ms/step - loss: 1.1601 - accuracy: 0.4771 - val_loss: 1.1477 -

In [6]:
model.summary()
model.evaluate(X_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (400, None, 100)          22900     
                                                                 
 simple_rnn_1 (SimpleRNN)    (400, 100)                20100     
                                                                 
 dropout (Dropout)           (400, 100)                0         
                                                                 
 dense (Dense)               (400, 4)                  404       
                                                                 
Total params: 43,404
Trainable params: 43,404
Non-trainable params: 0
_________________________________________________________________
13/13 [==============================] - 3s 182ms/step - loss: 1.1808 - accuracy: 0.4050


[1.1807944774627686, 0.4050000011920929]

## 2-1 LSTM

In [9]:
if 'model' in locals():
  del model
  gc.collect()
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [10]:
model = keras.models.Sequential([                      
    keras.layers.LSTM(150, return_sequences=True, dropout=.3),
    keras.layers.LSTM(150,  dropout=.1),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=4, activation='softmax')
])

In [8]:
model.compile(loss="sparse_categorical_crossentropy",
              #optimizer = keras.optimizers.Adam(learning_rate=0.001),
              optimizer ='rmsprop',
              metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=200, epochs=200,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])


Epoch 1/200
240/240 [==============================] - 22s 41ms/step - loss: 1.0336 - accuracy: 0.5455 - val_loss: 1.0782 - val_accuracy: 0.5175
Epoch 2/200
240/240 [==============================] - 7s 30ms/step - loss: 0.9061 - accuracy: 0.6221 - val_loss: 0.9692 - val_accuracy: 0.6275
Epoch 3/200
240/240 [==============================] - 7s 29ms/step - loss: 0.8306 - accuracy: 0.6618 - val_loss: 0.9219 - val_accuracy: 0.6300
Epoch 4/200
240/240 [==============================] - 7s 28ms/step - loss: 0.7799 - accuracy: 0.6864 - val_loss: 0.8511 - val_accuracy: 0.6500
Epoch 5/200
240/240 [==============================] - 7s 28ms/step - loss: 0.7380 - accuracy: 0.7063 - val_loss: 0.9796 - val_accuracy: 0.5875
Epoch 6/200
240/240 [==============================] - 7s 29ms/step - loss: 0.7068 - accuracy: 0.7204 - val_loss: 0.9298 - val_accuracy: 0.6425
Epoch 7/200
240/240 [==============================] - 7s 30ms/step - loss: 0.6751 - accuracy: 0.7345 - val_loss: 0.9296 - val_accuracy

In [9]:
model.summary()
model.evaluate(X_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (400, None, 150)          167400    
                                                                 
 lstm_1 (LSTM)               (400, 150)                180600    
                                                                 
 dropout (Dropout)           (400, 150)                0         
                                                                 
 dense (Dense)               (400, 4)                  604       
                                                                 
Total params: 348,604
Trainable params: 348,604
Non-trainable params: 0
_________________________________________________________________
13/13 [==============================] - 2s 64ms/step - loss: 1.2894 - accuracy: 0.5375


[1.2894359827041626, 0.5375000238418579]

## 2-2. GRU

In [11]:
if 'model' in locals():
  del model
  gc.collect()
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [13]:
model = keras.models.Sequential([
    keras.layers.GRU(100, return_sequences=True, dropout=.3),
    keras.layers.GRU(100, dropout=.1),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=4, activation='softmax')
])

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=200, epochs=200,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    

Epoch 1/200
240/240 [==============================] - 19s 28ms/step - loss: 1.0762 - accuracy: 0.5188 - val_loss: 0.9949 - val_accuracy: 0.6225
Epoch 2/200
240/240 [==============================] - 5s 19ms/step - loss: 0.9323 - accuracy: 0.6022 - val_loss: 0.8943 - val_accuracy: 0.6850
Epoch 3/200
240/240 [==============================] - 5s 23ms/step - loss: 0.8714 - accuracy: 0.6396 - val_loss: 0.8465 - val_accuracy: 0.7100
Epoch 4/200
240/240 [==============================] - 5s 19ms/step - loss: 0.8219 - accuracy: 0.6633 - val_loss: 0.8681 - val_accuracy: 0.7050
Epoch 5/200
240/240 [==============================] - 5s 19ms/step - loss: 0.7839 - accuracy: 0.6825 - val_loss: 0.8508 - val_accuracy: 0.7075
Epoch 6/200
240/240 [==============================] - 5s 20ms/step - loss: 0.7496 - accuracy: 0.6996 - val_loss: 0.8698 - val_accuracy: 0.7100
Epoch 7/200
240/240 [==============================] - 5s 22ms/step - loss: 0.7229 - accuracy: 0.7106 - val_loss: 0.9362 - val_accuracy

In [8]:
model.summary()
model.evaluate(X_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (200, None, 100)          69000     
                                                                 
 gru_1 (GRU)                 (200, 100)                60600     
                                                                 
 dropout (Dropout)           (200, 100)                0         
                                                                 
 dense (Dense)               (200, 4)                  404       
                                                                 
Total params: 130,004
Trainable params: 130,004
Non-trainable params: 0
_________________________________________________________________
13/13 [==============================] - 2s 59ms/step - loss: 1.2864 - accuracy: 0.6025


[1.2863861322402954, 0.6025000214576721]

## 3. Bidirectional GRU

In [18]:
if 'model' in locals():
  del model
  gc.collect()
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
model = keras.models.Sequential([
    keras.layers.Bidirectional(keras.layers.GRU(100, return_sequences=True, dropout=.3)),
    keras.layers.Bidirectional(keras.layers.GRU(100, dropout=.1)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=4, activation='softmax')
])


In [20]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=200, epochs=200,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

Epoch 1/200
240/240 [==============================] - 19s 55ms/step - loss: 1.0482 - accuracy: 0.5399 - val_loss: 0.8881 - val_accuracy: 0.6150
Epoch 2/200
240/240 [==============================] - 9s 38ms/step - loss: 0.8994 - accuracy: 0.6235 - val_loss: 0.8020 - val_accuracy: 0.6975
Epoch 3/200
240/240 [==============================] - 9s 38ms/step - loss: 0.8293 - accuracy: 0.6595 - val_loss: 0.8752 - val_accuracy: 0.6250
Epoch 4/200
240/240 [==============================] - 9s 38ms/step - loss: 0.7713 - accuracy: 0.6857 - val_loss: 0.8516 - val_accuracy: 0.6875
Epoch 5/200
240/240 [==============================] - 9s 38ms/step - loss: 0.7247 - accuracy: 0.7090 - val_loss: 0.8806 - val_accuracy: 0.6550
Epoch 6/200
240/240 [==============================] - 9s 39ms/step - loss: 0.6852 - accuracy: 0.7275 - val_loss: 0.9276 - val_accuracy: 0.6475
Epoch 7/200
240/240 [==============================] - 9s 37ms/step - loss: 0.6560 - accuracy: 0.7399 - val_loss: 0.9224 - val_accuracy

In [21]:
model.summary()
model.evaluate(X_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (200, None, 200)         138000    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (200, 200)               181200    
 nal)                                                            
                                                                 
 dropout (Dropout)           (200, 200)                0         
                                                                 
 dense (Dense)               (200, 4)                  804       
                                                                 
Total params: 320,004
Trainable params: 320,004
Non-trainable params: 0
_________________________________________________________________
13/13 [==============================] - 3s 87ms/s

[1.1772606372833252, 0.5425000190734863]

## 4. CNN

In [23]:
if 'model' in locals():
  del model
  gc.collect()
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [24]:
log_mels= np.load('/content/gdrive/My Drive/music_data_mining/log_mels_4classes.npz') 
X_train = log_mels['X_train']
X_test = log_mels['X_test']
X_valid = log_mels['X_valid']
y_train = log_mels['y_train']
y_test = log_mels['y_test']
y_valid = log_mels['y_valid']

In [25]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [26]:
print(X_train.shape, X_valid.shape, X_test.shape)

(3200, 1290, 128, 1) (400, 1290, 128, 1) (400, 1290, 128, 1)


In [27]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=10, kernel_size=5, input_shape=[ 1290, 128, 1], activation='relu', kernel_initializer='glorot_normal'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=10, kernel_size=5, activation='relu', kernel_initializer='glorot_normal'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=10, kernel_size=5, activation='relu', kernel_initializer='glorot_normal'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=10, kernel_size=5, activation='relu', kernel_initializer='glorot_normal'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=50, activation='relu', kernel_initializer='glorot_normal'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=20, activation='relu', kernel_initializer='glorot_normal'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=4, activation='softmax')
])

In [28]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

history = model.fit(X_train, y_train, batch_size=200, epochs=200,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
                    

Epoch 1/200
16/16 [==============================] - 15s 387ms/step - loss: 2.3106 - accuracy: 0.2553 - val_loss: 1.3721 - val_accuracy: 0.2800
Epoch 2/200
16/16 [==============================] - 6s 375ms/step - loss: 1.3693 - accuracy: 0.2753 - val_loss: 1.3261 - val_accuracy: 0.3425
Epoch 3/200
16/16 [==============================] - 6s 375ms/step - loss: 1.3433 - accuracy: 0.2966 - val_loss: 1.2862 - val_accuracy: 0.3400
Epoch 4/200
16/16 [==============================] - 6s 355ms/step - loss: 1.2908 - accuracy: 0.3241 - val_loss: 1.2478 - val_accuracy: 0.3325
Epoch 5/200
16/16 [==============================] - 6s 373ms/step - loss: 1.2747 - accuracy: 0.3578 - val_loss: 1.1409 - val_accuracy: 0.4350
Epoch 6/200
16/16 [==============================] - 6s 351ms/step - loss: 1.2272 - accuracy: 0.4059 - val_loss: 1.1710 - val_accuracy: 0.4700
Epoch 7/200
16/16 [==============================] - 6s 369ms/step - loss: 1.1974 - accuracy: 0.4322 - val_loss: 1.0934 - val_accuracy: 0.497

In [29]:
model.summary()
model.evaluate(X_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1286, 124, 10)     260       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 643, 62, 10)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 639, 58, 10)       2510      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 319, 29, 10)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 315, 25, 10)       2510      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 157, 12, 10)      0

[1.2198987007141113, 0.5099999904632568]